In [2]:
# load all libraries
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id

In [3]:
os.environ['AWS_ACCESS_KEY_ID']='ADD YOUR ACCESS KEY'
os.environ['AWS_SECRET_ACCESS_KEY']='ADD YOUR SECRET ACCESS KEY'

In [3]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID']) \
    .config("spark.hadoop.fs.s3a.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY']) \
    .getOrCreate()

sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("mapreduce.fileoutputcommitter.algorithm.version", "2")
sc._jsc.hadoopConfiguration().set("spark.speculation","false")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
schema =  StructType([
                        StructField("num_songs", ShortType(), True),
                        StructField("artist_latitude", StringType(), True),
                        StructField("artist_location", StringType(), True),
                        StructField("artist_longitude", StringType(), True),
                        StructField("artist_name", StringType(), True),
                        StructField("song_id", StringType(), False),
                        StructField("title", StringType(), True),
                        StructField("artist_id", StringType(), False),
                        StructField("year", ShortType(), True),
                        StructField("duration", DoubleType(), True)
                    ])

df_song_data = spark.read.schema(schema).json("s3a://udacity-dend/song_data/*/*/*/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
output_data = "hdfs:///dwh/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
df_song_data.write.mode('overwrite').parquet(output_data + "/songs_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
os.system("s3-dist-cp --src hdfs:///dwh/songs_data --dest s3a://udacity-data-engineering-stan/data/songs_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [36]:
songs_table = df_song_data.select("song_id", "title", "artist_id", "year", "duration")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
songs_table.write.mode('overwrite').partitionBy("year", "artist_id").parquet(output_data + "/songs_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
os.system("s3-dist-cp --src hdfs:///dwh/songs_table --dest s3a://udacity-data-engineering-stan/data/songs_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [39]:
artists_table = df_song_data.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
artists_table.write.mode('overwrite').parquet(output_data + "/artists_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
os.system("s3-dist-cp --src hdfs:///dwh/artists_table --dest s3a://udacity-data-engineering-stan/data/artists_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [88]:
# create schema and read log data file
    
schema =  StructType([
                        StructField("artist", StringType(), True),
                        StructField("auth", StringType(), True),
                        StructField("firstName", StringType(), True),
                        StructField("gender", StringType(), True),
                        StructField("itemInSession", ShortType(), True),
                        StructField("lastName", StringType(), True),
                        StructField("length", StringType(), True),
                        StructField("level", StringType(), True),
                        StructField("location", StringType(), True),
                        StructField("method", StringType(), True),
                        StructField("page", StringType(), True),
                        StructField("registration", StringType(), True),
                        StructField("sessionId", ShortType(), False),
                        StructField("song", StringType(), True),
                        StructField("status", StringType(), True),
                        StructField("ts", DoubleType(), True),
                        StructField("userAgent", StringType(), True),
                        StructField("userId", StringType(), False)
                    ])

df_log_data = spark.read.schema(schema).json("s3a://udacity-dend/log_data/*/*/*.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
df_log_data = df_log_data.where(df_log_data.page == "NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df_log.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|               ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+
|            Harmonia| Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|       Sehr kosmisch|   200|1.542241826796E12|"Mozilla/5.0 (X11...|    26|
|         The Prodigy| Logged In|     Ryan|     M|            1|   Smith|260.07465| 

In [93]:
users_table = df_log_data.select("userId", "firstName", "lastName", "gender", "level").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
users_table.write.mode('overwrite').partitionBy("gender", "level").parquet(output_data + "/users_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
os.system("s3-dist-cp --src hdfs:///dwh/users_table --dest s3a://udacity-data-engineering-stan/data/users_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [96]:
get_timestamp = udf(lambda x: datetime.fromtimestamp((int(x)/1000.0)), TimestampType())
df_log_data = df_log_data.withColumn('timestamp', get_timestamp(df_log_data.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
df_log_data = df_log_data.withColumn('start_time', date_format("timestamp", 'HH:mm:ss'))\
        .withColumn('hour', hour("timestamp"))\
        .withColumn('day', dayofmonth("timestamp"))\
        .withColumn('week', weekofyear("timestamp"))\
        .withColumn('month', month("timestamp"))\
        .withColumn('year', year("timestamp"))\
        .withColumn('weekday', date_format('timestamp', 'E'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
df_log_data.write.mode('overwrite').parquet(output_data + "/log_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
os.system("s3-dist-cp --src hdfs:///dwh/log_data --dest s3a://udacity-data-engineering-stan/data/log_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [98]:
time_table = df_log_data.select("start_time", "hour", "day", "week", "month", "year", "weekday").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
time_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "/time_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
os.system("s3-dist-cp --src hdfs:///dwh/time_table --dest s3a://udacity-data-engineering-stan/data/time_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [129]:
songs_df = spark.read.parquet(output_data + 'songs_data')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
songs_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+--------------------+----------------+--------------------+------------------+--------------------+------------------+----+----------+
|num_songs|artist_latitude|     artist_location|artist_longitude|         artist_name|           song_id|               title|         artist_id|year|  duration|
+---------+---------------+--------------------+----------------+--------------------+------------------+--------------------+------------------+----+----------+
|        1|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|SOVIYJY12AF72A4B00|The Dead Next Doo...|AR4T2IF1187B9ADBB7|1983| 233.22077|
|        1|       63.96027|<a href="http://b...|        10.22442|          Billy Idol|SOVYXYL12AF72A3373|Rebel Yell (1999 ...|AR4T2IF1187B9ADBB7|1983| 287.92118|
|        1|           null|                    |            null|Yvonne S. Moriart...|SOEPTVC12A67ADD0DA|To Zucchabar ["Gl...|ARQ846I1187B9A7083|   0| 196.04853|
|        1|       63.96027|<

In [ ]:
songs_df_small = songs_df.select("num_songs", "title", "artist_id", "duration", "artist_name", "song_id")

In [115]:
log_df = spark.read.parquet(output_data + 'log_data/*')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
log_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+--------------------+----------+----+---+----+-----+----+-------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|               ts|           userAgent|userId|           timestamp|start_time|hour|day|week|month|year|weekday|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+--------------------+----------+----+---+----+-----+----+-------+
|            Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextS

In [131]:
df_join = songs_df_small.join(log_df, songs_df_small.title == log_df.song)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
df_join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+--------------------+----------------+--------------------+------------------+--------------------+------------------+----+---------+--------------------+---------+----------+------+-------------+---------+---------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-----------------+--------------------+------+--------------------+----------+----+---+----+-----+----+-------+
|num_songs|artist_latitude|     artist_location|artist_longitude|         artist_name|           song_id|               title|         artist_id|year| duration|              artist|     auth| firstName|gender|itemInSession| lastName|   length|level|            location|method|    page|     registration|sessionId|                song|status|               ts|           userAgent|userId|           timestamp|start_time|hour|day|week|month|year|weekday|
+---------+---------------+--------------------+----------------+--------------------+------

In [133]:
songplays_table = df_join.select("start_time",
                                 "userId",
                                 "level",
                                 "song_id",
                                 "artist_id",
                                 "sessionId",
                                 "location",
                                 "userAgent",
                                 "year",
                                 "month").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
songplays_table = songplays_table.withColumn("songplay_id", monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [135]:
songplays_table = songplays_table.select("songplay_id",
                                             "start_time",
                                             "userId",
                                             "level",
                                             "song_id",
                                             "artist_id",
                                             "sessionId",
                                             "location",
                                             "userAgent",
                                             "year",
                                             "month")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
songplays_table.write.mode('overwrite').partitionBy("year", "month").parquet(output_data + "/songplays_table/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
os.system("s3-dist-cp --src hdfs:///dwh/songplays_table --dest s3a://udacity-data-engineering-stan/data/songplays_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0